In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import $ivy.`org.apache.spark::spark-mllib:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$
import $ivy.$
import org.apache.log4j.{Level, Logger}

# Spark ML Production

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Production")
                .config("spark.sql.debug.maxToStringFields", 1024)
                .getOrCreate()

import spark.implicits._

spark.sparkContext.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/07/16 14:30:22 INFO SparkContext: Running Spark version 3.5.0
24/07/16 14:30:22 INFO SparkContext: OS info Linux, 6.5.0-41-generic, amd64
24/07/16 14:30:22 INFO SparkContext: Java version 11.0.23
24/07/16 14:30:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/16 14:30:23 INFO ResourceUtils: ==============================================================
24/07/16 14:30:23 INFO ResourceUtils: No custom resources configured for spark.driver.
24/07/16 14:30:23 INFO ResourceUtils: ==============================================================
24/07/16 14:30:23 INFO SparkContext: Submitted application: Spark ML Production
24/07/16 14:30:23 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , 

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: SparkSession = org.apache.spark.sql.SparkSession@4af40d58
import spark.implicits._

## Загружаем данные

In [3]:
val data = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(s"../../data/BankChurners.csv")

data: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]

## Загружаем модель

In [4]:
import org.apache.spark.ml.PipelineModel

val model = PipelineModel.load(s"pipelineModel")

import org.apache.spark.ml.PipelineModel
model: PipelineModel = pipeline_b854e5ef5905

## Вычисляем

In [5]:
val predicted = model.transform(data)

predicted: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 37 more fields]

In [6]:
predicted.show(10)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

## Проверяем результат

In [7]:
val tp = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 1)).count()
val tn = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 0)).count()
val fp = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 1)).count()
val fn = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 0)).count()

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t\t$tn\n")

Confusion Matrix:
1205	1898
422		6602



tp: Long = 1205L
tn: Long = 6602L
fp: Long = 1898L
fn: Long = 422L

In [8]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.7709094499851881
Precision = 0.3883338704479536
Recall = 0.7406269207129687



accuracy: Double = 0.7709094499851881
precision: Double = 0.3883338704479536
recall: Double = 0.7406269207129687